In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
pk = pickle.load(open('/home/wzc26/work/vcc/nbglm/data/true_de_cache_all.pkl', 'rb'))

In [5]:
pk.keys()

dict_keys(['ACAT2', 'ACVR1B', 'AKT2', 'ANTXR1', 'ARID1A', 'ATM', 'ATP1B1', 'ATP6V0B', 'ATP6V0C', 'BIRC2', 'BRD9', 'C1QBP', 'CALM3', 'CAMSAP2', 'CASP2', 'CASP3', 'CAST', 'CDCA2', 'CENPO', 'CHMP3', 'CLDN6', 'CLDN7', 'CREG1', 'DAXX', 'DHCR24', 'DHX36', 'DLG5', 'DNAJA3', 'DNMT1', 'DZIP3', 'EID2', 'EIF4B', 'EPB41L2', 'EPHB4', 'ETV4', 'EWSR1', 'FDPS', 'FOXH1', 'GSK3B', 'HAT1', 'HIRA', 'HMGB2', 'HMGN1', 'HSBP1', 'IDE', 'IGF2R', 'IKBKG', 'INSIG1', 'IRF3', 'KAT2A', 'KDM1A', 'KDM2B', 'KDR', 'KIF1B', 'KIF20A', 'KLF10', 'LAD1', 'LRPPRC', 'LZTR1', 'MAP1B', 'MAP3K7', 'MAPKAPK2', 'MAST2', 'MAU2', 'MBTPS1', 'MED1', 'MED12', 'MED13', 'MED13L', 'MED24', 'METTL14', 'METTL17', 'METTL3', 'MKI67', 'MTA1', 'NCK2', 'NDUFB4', 'NDUFB6', 'NISCH', 'NIT1', 'NRAS', 'NREP', 'OXA1L', 'OXCT1', 'PAGR1', 'PBX1', 'PHF10', 'PHF14', 'PLCB3', 'PMEL', 'PMS1', 'POLB', 'PRCP', 'PRDM14', 'PRR12', 'PTPN1', 'RAB3B', 'RAF1', 'RNF2', 'RNF20', 'RRM1', 'SAFB', 'SALL4', 'SDC1', 'SEC62', 'SHPRH', 'SIN3B', 'SIX4', 'SLC25A3', 'SLC39A6', 

In [6]:
pk['ACAT2']

,names,pvals_adj,logfoldchanges
0,MVD,0.0,1.161838
1,HMGCS1,0.0,1.064512
2,DHCR24,0.0,0.539191
3,CYB5B,0.0,0.631912
4,HMGCR,0.0,0.850397
...,...,...,...
18075,EXD2,0.0,-1.510558
18076,RCN1,0.0,-0.619534
18077,PLCB4,0.0,-2.251294
18078,SP1,0.0,-2.135239


In [16]:
import anndata as ad
import numpy as np
import pandas as pd

from pdex import parallel_differential_expression

PERT_COL = "perturbation"
CONTROL_VAR = "control"

N_CELLS = 30000
N_GENES = 20000
N_PERTS = 30
MAX_UMI = 1e6


def build_random_anndata(
    n_cells: int = N_CELLS,
    n_genes: int = N_GENES,
    n_perts: int = N_PERTS,
    pert_col: str = PERT_COL,
    control_var: str = CONTROL_VAR,
) -> ad.AnnData:
    """Sample a random AnnData object."""
    return ad.AnnData(
        X=np.random.randint(0, MAX_UMI, size=(n_cells, n_genes)),
        obs=pd.DataFrame(
            {
                pert_col: np.random.choice(
                    [f"pert_{i}" for i in range(n_perts)] + [control_var],
                    size=n_cells,
                    replace=True,
                ),
            }
        ),
    )
adata = build_random_anndata()

/mnt/nvme0n1/users/wzc26/conda/envs/torch/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


In [26]:

# Run pdex with default metric (wilcoxon)
results = parallel_differential_expression(
    adata,
    reference=CONTROL_VAR,
    groupby_key=PERT_COL,
    num_workers=128,
    batch_size=2048
)
assert results.shape[0] == N_GENES * N_PERTS

INFO:pdex._single_cell:Precomputing masks for each target gene
Identifying target masks: 100%|██████████| 31/31 [00:00<00:00, 578.61it/s]
INFO:pdex._single_cell:Precomputing variable indices for each feature
Identifying variable indices: 100%|██████████| 20000/20000 [00:00<00:00, 3051734.58it/s]
INFO:pdex._single_cell:Creating shared memory memory matrix for parallel computing
INFO:pdex._single_cell:Creating generator of all combinations: N=620000
INFO:pdex._single_cell:Creating generator of all batches: N=303
INFO:pdex._single_cell:Initializing parallel processing pool
INFO:pdex._single_cell:Processing batches
Processing batches: 100%|██████████| 303/303 [00:11<00:00, 26.26it/s]
INFO:pdex._single_cell:Flattening results
INFO:pdex._single_cell:Closing shared memory pool


In [14]:
adata.obs[PERT_COL].value_counts()

perturbation
pert_0     2504
control    2496
Name: count, dtype: int64

In [ ]:
results

,target,reference,feature,target_mean,reference_mean,percent_change,fold_change,p_value,statistic,fdr
0,pert_19,control,0,492140.665641,510253.149551,-0.035497,0.964503,0.175336,471751.0,0.999825
1,pert_19,control,1,504605.696410,519140.020937,-0.027997,0.972003,0.270832,474978.0,0.999825
2,pert_19,control,2,502014.613333,496343.506481,0.011426,1.011426,0.665150,494459.5,0.999825
3,pert_19,control,3,502184.872821,505266.019940,-0.006098,0.993902,0.815423,485997.5,0.999825
4,pert_19,control,4,507125.008205,504876.007976,0.004455,1.004455,0.859867,491205.0,0.999825
...,...,...,...,...,...,...,...,...,...,...
599995,pert_3,control,19995,504783.852356,504257.417747,0.001044,1.001044,0.963006,479513.0,0.999825
599996,pert_3,control,19996,504448.621990,510659.027916,-0.012162,0.987838,0.634320,472984.0,0.999825
599997,pert_3,control,19997,498654.113089,484450.855434,0.029318,1.029318,0.281864,492390.0,0.999825
599998,pert_3,control,19998,519737.270157,502279.900299,0.034756,1.034756,0.159269,496534.0,0.999825
